# Pytest Runner — Запуск тестов из Jupyter

Этот ноутбук помогает удобно запускать тесты проекта с помощью `pytest` прямо из ячеек.

Инструкция:
- Выполняйте ячейки сверху вниз.
- В секциях есть готовые функции-обёртки: запуск всех тестов, запуск по файлу, запуск по nodeid, запуск по маркеру.
- Отчёт в формате JSON сохраняется в `pytest_report.json` и копируется в папку `reports/` с меткой времени.


## 1) Установить зависимости

Выполните эту ячейку, если нужно установить `pytest` и плагин для JSON-отчётов. Это опционально, если уже установлено в окружении.


In [9]:
# Установка зависимостей (опционально)
# Запустите, если нужно установить плагины в этом окружении.
# !pip install pytest pytest-json-report


## 2) Импорт библиотек и глобальные переменные

Этот блок устанавливает пути, глобальные переменные и печатает базовую информацию об окружении.


In [ ]:
# Импорт и глобальные переменные
from pathlib import Path
import sys
import subprocess
import json
from datetime import datetime
from IPython.display import display, HTML, Markdown

# Если ноутбук находится в tests/, поднимаемся на уровень выше
NOTEBOOK_DIR = Path('.').resolve()
if NOTEBOOK_DIR.name == 'tests':
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    PROJECT_ROOT = NOTEBOOK_DIR

REPORT_FILE = PROJECT_ROOT / 'pytest_report.json'
REPORTS_DIR = PROJECT_ROOT / 'reports'
REPORTS_DIR.mkdir(exist_ok=True)

print(f'Project root: {PROJECT_ROOT}')
print(f'Notebook dir: {NOTEBOOK_DIR}')
print(sys.version)

# Попробуем показать версии pytest
try:
    import pytest
    print('pytest version:', pytest.__version__)
except Exception:
    print('pytest не найден в текущем окружении')


## 3) Показать и сохранить JSON/HTML отчёт

Эта секция читает `pytest_report.json`, показывает краткую сводку и сохраняет копию в `reports/`.


In [11]:
import re

def remove_ansi_codes(text):
    """Удаляет ANSI-коды цветов из текста."""
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

def summarize_report(output_text, returncode):
    """
    Анализирует вывод pytest и показывает краткую сводку.
    
    Аргументы:
    - output_text: вывод из stdout/stderr pytest (может содержать ANSI-коды)
    - returncode: код возврата процесса
    """
    # Удаляем ANSI-коды из текста
    clean_output = remove_ansi_codes(output_text)
    lines = clean_output.split('\n')
    
    # Ищем строку с итогами (pytest выводит что-то вроде "24 passed in 15.85s")
    summary_line = None
    for line in lines:
        line_lower = line.lower()
        if any(keyword in line_lower for keyword in ['passed', 'failed', 'error', 'skipped']):
            if 'in ' in line_lower and ('s' in line_lower or 'ms' in line_lower):
                summary_line = line.strip()
                break
    
    md = f"### pytest — результат\n\n"
    if returncode == 0:
        md += "✅ **Все тесты пройдены!**\n"
    else:
        md += "❌ **Некоторые тесты упали или произошла ошибка!**\n"
    
    if summary_line:
        md += f"\n```\n{summary_line}\n```\n"
    
    md += f"\nКод возврата: {returncode}\n"
    
    display(Markdown(md))
    
    # Сохраняем лог (без ANSI-кодов для читаемости)
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_path = REPORTS_DIR / f'pytest_log_{ts}.txt'
    log_path.write_text(clean_output, encoding='utf-8')
    print(f'✓ Лог сохранён в {log_path}')


## 4) Создать/обновить `pytest.ini`

Этот блок создаёт или перезаписывает `pytest.ini` в корне проекта с базовыми настройками.


In [ ]:
# Запись pytest.ini
pytest_ini = PROJECT_ROOT / 'pytest.ini'
pytest_ini_content = """[pytest]
addopts = --maxfail=1 -q
"""
pytest_ini.write_text(pytest_ini_content, encoding='utf-8')
print(f'Записан {pytest_ini}')


## 5) Утилита run_pytests (запуск и разбор JSON-отчёта)

Этот блок содержит функцию `run_pytests`, которая запускает `pytest` через `subprocess`, стримит вывод и собирает JSON-отчёт.


In [13]:
import os

def run_pytests(args=None, cwd=None, timeout=None):
    """
    Запускает pytest и возвращает вывод.

    Аргументы:
    - args: список аргументов для pytest (например, ['tests/unit'])
    - cwd: рабочая директория (по умолчанию PROJECT_ROOT)
    - timeout: таймаут процесса в секундах
    
    Возвращает: (output_text, returncode)
    """
    if args is None:
        args = []
    if cwd is None:
        cwd = PROJECT_ROOT
    else:
        cwd = Path(cwd)

    # Простая команда pytest без специальных плагинов
    cmd = [sys.executable, '-m', 'pytest', '-v'] + args
    print('📝 Выполняется команда:')
    print('  ' + ' '.join(map(str, cmd)))
    print()

    proc = subprocess.Popen(
        cmd,
        cwd=str(cwd),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )

    out_lines = []
    try:
        for line in proc.stdout:
            out_lines.append(line)
            print(line, end='')
        proc.wait(timeout=timeout)
    except Exception as e:
        proc.kill()
        print(f'Ошибка при выполнении pytest: {e}')
        raise

    rc = proc.returncode
    return ''.join(out_lines), rc


## 6) Запуск всех тестов

Функция `run_all_tests()` выполняет все тесты в проекте и выводит краткую сводку.


In [ ]:
def run_all_tests():
    """Запускает все тесты в проекте."""
    output, code = run_pytests(['.'])
    summarize_report(output, code)
    return code

# Пример вызова (раскомментируйте для запуска прямо в ноутбуке):
run_all_tests()


## 7) Запуск тестов для текущего файла (переменная TEST_FILE)

Укажите `TEST_FILE` — путь к тестовому файлу, например `tests/unit/test_constants.py`, и вызовите `run_test_file(TEST_FILE)`.


In [7]:
TEST_FILE = ''  # Пример: 'tests/unit/test_constants.py'

def run_test_file(test_file):
    """Запускает тесты из конкретного файла."""
    output, code = run_pytests([test_file])
    summarize_report(output, code)
    return code

# Пример:
run_test_file('tests/unit/test_constants.py')


📝 Выполняется команда:
  c:\ProgramData\anaconda3\python.exe -m pytest -v tests/unit/test_constants.py

============================= test session starts =============================
platform win32 -- Python 3.13.5, pytest-8.3.4, pluggy-1.5.0
rootdir: D:\Программирование\Fork\Scientific_research_osc_ML
configfile: pytest.ini
plugins: anyio-4.7.0
collected 18 items

tests\unit\test_constants.py ..................                          [100%]

============================= 18 passed in 0.04s ==============================


### pytest — результат

✅ **Все тесты пройдены!**

[32m============================= [32m[1m18 passed[0m[32m in 0.04s[0m[32m ==============================[0m

Код возврата: 0


✓ Лог сохранён в D:\Программирование\Fork\Scientific_research_osc_ML\reports\pytest_log_20251121_000120.txt


0

## 8) Запуск отдельного теста по nodeid или по ключевому слову

Пример: `run_test_node('tests/unit/test_pdr_calculator.py::TestSymmetricalComponents::test_balanced_system')`


In [ ]:
def run_test_node(nodeid):
    """Запускает конкретный тест по его nodeid."""
    output, code = run_pytests([nodeid])
    summarize_report(output, code)
    return code


def run_by_keyword(keyword):
    """Запускает тесты, соответствующие ключевому слову."""
    output, code = run_pytests(['-k', keyword])
    summarize_report(output, code)
    return code

# Примеры:
# run_test_node('tests/unit/test_pdr_calculator.py::TestSymmetricalComponents::test_balanced_system')
# run_by_keyword('unit and not slow')


## 9) Поведение при ошибках и код возврата для CI / VSCode

Если тесты упали, используйте `run_and_raise()` для получения ненулевого кода возврата (можно использовать в CI).


In [ ]:
def run_and_raise(args=None):
    """
    Запускает тесты и вызывает SystemExit если они упали.
    Используется для CI.
    """
    output, code = run_pytests(args)
    summarize_report(output, code)
    if code != 0:
        raise SystemExit(code)
    return code

# Пример использования в CI:
# run_and_raise(['tests/unit'])


## Утилиты: Просмотр тестовых файлов и быстрые примеры

Функция `list_test_files()` покажет все найденные тест-файлы.


In [6]:
def list_test_files():
    files = sorted(PROJECT_ROOT.glob('tests/**/*.py'))
    for p in files:
        print(p)
    return files

print('\nПримеры команд (выполните в ячейке):')
print("- run_all_tests()")
print("- run_test_file('tests/unit/test_constants.py')")
print("- run_test_node('tests/unit/test_pdr_calculator.py::TestSymmetricalComponents::test_balanced_system')")
print("- run_by_keyword('unit and not slow')")



Примеры команд (выполните в ячейке):
- run_all_tests()
- run_test_file('tests/unit/test_constants.py')
- run_test_node('tests/unit/test_pdr_calculator.py::TestSymmetricalComponents::test_balanced_system')
- run_by_keyword('unit and not slow')
